In [1]:
from pyspark import SparkConf, SparkContext
import sys

conf = SparkConf().setAppName("Spark Application")
sc = SparkContext(conf=conf)

In [99]:
threshold = 0.0

# movieid, movie-genre
moviesRDD = sc.textFile("movies.txt").map(lambda value: (value.split(",")[0], value.split(",")[2])) 

# userid,movie-genre
preferencesRDD = sc.textFile("preferences.txt").map(lambda value: (value.split(",")[0], value.split(",")[1]))

# (movieid,userid)
watchedMoviesRDD = sc.textFile("watchedmovies.txt").map(lambda value: (value.split(",")[1], value.split(",")[0])) 

# (userid, [watched movie-genres])
userMovieGenreRDD = watchedMoviesRDD.join(moviesRDD)\
    .map(lambda pair: (pair[1][0], pair[1][1]))\
    .groupByKey()

userPreferencesMovieGenreRDD = preferencesRDD.groupByKey() # (userid, [preferred movie-genres])

# ( userid, ([not distinct movie genres], [ preferred movie genres]))
userMovieGenres = userPreferencesMovieGenreRDD.cogroup(userMovieGenreRDD) 
    
users = userMovieGenres.filter(isUserMisleading).keys()
        
for user in users.collect():
    print(user)

user2
user1


In [100]:
def isUserMisleading(pair):
    threshold = 0.5
    
    userId = pair[0]
    listPreferenceMovieGenres = list(pair[1][0]) #list
    listWatchedMovieGenres = list(pair[1][1]) #list

    numberOfMoviesSeen = len(watchedMovieGenres)
    
    # number of films with movie genres not included in preferred movie genres / number of films seen
    notPreferredGenresFilms = 0
    
    for watchedMovieGenres in listWatchedMovieGenres:
        for movieGenre in watchedMovieGenres:
            if movieGenre not in preferenceMovieGenres:
                notPreferredGenresFilms += 1
    
    proportion = notPreferredGenresFilms / numberOfMoviesSeen
    
    if (proportion > threshold):
        return True
    else:
        return False
    
    
    